In [1]:
import numpy as np
import pandas as pd
import h5py
import pysam
import os
from modisco.visualization import viz_sequence
from modisco import util
from matplotlib import pyplot as plt
import pybedtools
from bs4 import BeautifulSoup

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
dttype = 'DNASE'
mode = "counts"
celltype="IMR90"
modisco_path = '/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/'+dttype+'/'+celltype+'/merge_folds_new/'+mode+'/modisco_old_format.h5'
ppm_dir = ''

htmld="/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/"+dttype+'/'+celltype+"/merge_folds_new/"+mode+"/motifs.html"
tomtom = pd.read_html(htmld)


background=[0.25, 0.25, 0.25, 0.25]

In [3]:
def trim_motif_new(cwm, motif, trim_threshold=0.20):
    """
    Given the PFM and motif (both L x 4 arrays) (the motif could be the
    PFM itself), trims `motif` by cutting off flanks of low information
    content in `pfm`. `min_ic` is the minimum required information
    content. If specified this trimmed motif will be extended on either
    side by `pad` bases.
    If no base passes the `min_ic` threshold, then no trimming is done.
    """
    
    score = np.sum(np.abs(cwm), axis=1)
    trim_thresh = np.max(score) * trim_threshold  # Cut off anything less than 30% of max score
    pass_inds = np.where(score >= trim_thresh)[0]
    trimmed = motif[np.min(pass_inds): np.max(pass_inds) + 1]
 
    if not trimmed.size:
        return motif
    
    return trimmed

def import_tfmodisco_motifs(tfm_results_path, trim=True, only_pos=True):
    """
    Imports the PFMs to into a dictionary, mapping `(x, y)` to the PFM,
    where `x` is the metacluster index and `y` is the pattern index.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `out_dir`: where to save motifs
        `trim`: if True, trim the motif flanks based on information content
        `only_pos`: if True, only return motifs with positive contributions
    Returns the dictionary of PFMs.
    """ 
    pfms = {}
    with h5py.File(tfm_results_path, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if "patterns" not in metacluster["seqlets_to_patterns_result"]:
                continue
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
#                pattern_name = pattern_name.decode()
                pattern_name = pattern_name

                pattern = patterns[pattern_name]
                pfm = pattern["sequence"]["fwd"][:]
                cwm = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if only_pos and np.sum(cwm) < 0:
                    continue
                    
                if trim:
                    pfm = trim_motif_new(cwm, cwm)
                else:
                    pfm = cwm
                    
                pfms["%d_%d" % (metacluster_i,pattern_i)] = pfm
    return pfms

In [4]:
pfms = import_tfmodisco_motifs(modisco_path, trim=False)

In [5]:
for key in pfms:
    f = open(os.path.join(ppm_dir,mode+"_"+key+".pfm"),"w")
    #print(pfms[key])
    np.savetxt(f, pfms[key], fmt='%f')
    f.close()
    

In [6]:
tomtom[0]

,pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
0,pos_patterns.pattern_0,81493,NaN,NaN,FOSL1+JUN_MA1128.1,5.955930e-04,NaN,FOS+JUND_MA1141.1,5.955930e-04,NaN,ATF3_MOUSE.H11MO.0.A,1.746280e-03,NaN
1,pos_patterns.pattern_1,21717,NaN,NaN,CTCF_MA0139.1,9.842360e-12,NaN,CTCF_HUMAN.H11MO.0.A,3.699040e-09,NaN,CTCF_MOUSE.H11MO.0.A,4.835700e-08,NaN
2,pos_patterns.pattern_2,12484,NaN,NaN,NFIC_HUMAN.H11MO.0.A,1.617250e-03,NaN,NFIA_HUMAN.H11MO.0.C,7.790550e-02,NaN,NFIA_MOUSE.H11MO.0.C,7.790550e-02,NaN
3,pos_patterns.pattern_3,9546,NaN,NaN,TEAD3_MA0808.1,1.646020e-04,NaN,TEAD3_TEA_2,1.646020e-04,NaN,TEAD1_MOUSE.H11MO.0.A,2.200090e-02,NaN
4,pos_patterns.pattern_4,7776,NaN,NaN,RUNX1_HUMAN.H11MO.0.A,1.512400e-02,NaN,RUNX1_MOUSE.H11MO.0.A,1.512400e-02,NaN,RUNX3_MOUSE.H11MO.0.A,1.512400e-02,NaN
5,pos_patterns.pattern_5,7181,NaN,NaN,Foxc1.mouse_forkhead_2,2.195140e-02,NaN,FOXJ3_forkhead_2,2.195140e-02,NaN,FOXG1_forkhead_1,2.195140e-02,NaN
6,pos_patterns.pattern_6,6867,NaN,NaN,JUN_MA0488.1,1.376090e-03,NaN,JUND_MA0492.1,3.974990e-03,NaN,ATF2_HUMAN.H11MO.0.B,3.974990e-03,NaN
7,pos_patterns.pattern_7,6667,NaN,NaN,FOSL1+JUND_MA1143.1,1.502480e-03,NaN,Atf1_MA0604.1,2.745420e-03,NaN,Crem_MA0609.1,5.097960e-02,NaN
8,pos_patterns.pattern_8,5459,NaN,NaN,FOSL2_MA0478.1,1.576570e-02,NaN,JUNB_MA0490.1,1.576570e-02,NaN,BATF+JUN_MA0462.1,1.996820e-02,NaN
9,pos_patterns.pattern_9,4696,NaN,NaN,ATF4_MOUSE.H11MO.0.A,5.491060e-04,NaN,ATF4_HUMAN.H11MO.0.A,7.297840e-04,NaN,DDIT3_MOUSE.H11MO.0.C,7.297840e-04,NaN


In [7]:

tomtom[0]["pattern"] = tomtom[0]["pattern"].str.replace("pos_patterns.pattern","0").str.replace("neg_patterns.pattern","1")


/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [8]:
tomtom[0]

,pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
0,0_0,81493,NaN,NaN,FOSL1+JUN_MA1128.1,5.955930e-04,NaN,FOS+JUND_MA1141.1,5.955930e-04,NaN,ATF3_MOUSE.H11MO.0.A,1.746280e-03,NaN
1,0_1,21717,NaN,NaN,CTCF_MA0139.1,9.842360e-12,NaN,CTCF_HUMAN.H11MO.0.A,3.699040e-09,NaN,CTCF_MOUSE.H11MO.0.A,4.835700e-08,NaN
2,0_2,12484,NaN,NaN,NFIC_HUMAN.H11MO.0.A,1.617250e-03,NaN,NFIA_HUMAN.H11MO.0.C,7.790550e-02,NaN,NFIA_MOUSE.H11MO.0.C,7.790550e-02,NaN
3,0_3,9546,NaN,NaN,TEAD3_MA0808.1,1.646020e-04,NaN,TEAD3_TEA_2,1.646020e-04,NaN,TEAD1_MOUSE.H11MO.0.A,2.200090e-02,NaN
4,0_4,7776,NaN,NaN,RUNX1_HUMAN.H11MO.0.A,1.512400e-02,NaN,RUNX1_MOUSE.H11MO.0.A,1.512400e-02,NaN,RUNX3_MOUSE.H11MO.0.A,1.512400e-02,NaN
5,0_5,7181,NaN,NaN,Foxc1.mouse_forkhead_2,2.195140e-02,NaN,FOXJ3_forkhead_2,2.195140e-02,NaN,FOXG1_forkhead_1,2.195140e-02,NaN
6,0_6,6867,NaN,NaN,JUN_MA0488.1,1.376090e-03,NaN,JUND_MA0492.1,3.974990e-03,NaN,ATF2_HUMAN.H11MO.0.B,3.974990e-03,NaN
7,0_7,6667,NaN,NaN,FOSL1+JUND_MA1143.1,1.502480e-03,NaN,Atf1_MA0604.1,2.745420e-03,NaN,Crem_MA0609.1,5.097960e-02,NaN
8,0_8,5459,NaN,NaN,FOSL2_MA0478.1,1.576570e-02,NaN,JUNB_MA0490.1,1.576570e-02,NaN,BATF+JUN_MA0462.1,1.996820e-02,NaN
9,0_9,4696,NaN,NaN,ATF4_MOUSE.H11MO.0.A,5.491060e-04,NaN,ATF4_HUMAN.H11MO.0.A,7.297840e-04,NaN,DDIT3_MOUSE.H11MO.0.C,7.297840e-04,NaN


In [9]:
tomtom[0][["pattern","num_seqlets"]].to_csv(os.path.join(ppm_dir,mode+"_counts.csv"),sep=",",index=False, header=False)